In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product



options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [2]:
K = 3
lambda_star = np.array([-1,3,1])

I = 25
J = 40
# sigma = 1

### generate exogenous data

# np.random.seed(2)
# φ_i_j_k = np.random.normal(0,1, size=[I,J,K])
# # eps_ij = sigma * np.random.normal(0,1, size = [I,J])
# def φ_k(i_idx,bundles):
#     return (φ_i_j_k[i_idx[:,None], bundles,:]).sum(1)



np.random.seed(4)
φ_i_j_k = np.random.normal(0,5, size=[I,J,K-1])
# eps_i_j = sigma * np.random.normal(0,1, size = [I,J])

phi_j = np.random.normal(0,1, size=[J])**2
t_j_j = np.random.normal(0,1, size=[J,J])**2
t_j_sum = t_j_j.sum(1)
def φ_k(i_idx,bundles):
    complementarities_i_k = (phi_j[bundles] * (t_j_j[bundles[:,:,None] , bundles[:,None,:]]).sum(-1) 
                             / t_j_sum[bundles]).sum(-1)
    
    return np.hstack(((φ_i_j_k[i_idx[:,None], bundles,:]).sum(1), complementarities_i_k.reshape(-1, 1) ))

In [3]:
def grad_lovatz_extension(i_idx, z_i_j, lambda_k, eps_i_j = None) :
    if eps_i_j is None:
        eps_i_j = np.zeros([len(i_idx),J])              

    sorted_z_id_j = z_i_j.argsort(1)[:,::-1]

    grad = np.zeros([len(z_i_j),J])
    for j in range(J):
        grad[np.arange(len(i_idx)),sorted_z_id_j[:,j]] = (φ_k(i_idx, sorted_z_id_j[:,np.arange(j+1)] ) @ lambda_k 
                                    - φ_k(i_idx, sorted_z_id_j[:,np.arange(j)] ) @ lambda_k  
                                    + eps_i_j[np.arange(len(i_idx)),sorted_z_id_j[:,j]])
    
    return grad


def maximize_supermodular(i_idx, lambda_k , num_iterations,alpha ,eps_i_j = None):

    z_t = np.ones([len(i_idx),J])/2 

    z_list = []
    iter = 0 
    for _ in range(num_iterations):
        grad = grad_lovatz_extension(i_idx, z_t, lambda_k, eps_i_j) 
        z_new = z_t + alpha * grad / np.linalg.norm(grad)
        z_new = np.clip(z_new, 0, 1)

        z_t = z_new
        z_list.append(z_new)
        iter += 1

    z_star = np.array(z_list)[-int(np.floor(num_iterations)/10):,:,:].mean(0)
    bundle_star = np.array(z_star.round(0), dtype= bool)

    return bundle_star, z_star

In [4]:
# def maximize_naive(i_idx, lambda_k  ,eps_i_j = None):
#     if eps_i_j is None:
#         eps_i_j = np.zeros([len(i_idx),J]) 
    
#     bundle_star = (φ_i_j_k[i_idx] @ lambda_star + eps_i_j) > 1e-13
#     print( np.max(φ_i_j_k[i_idx] @ lambda_star + eps_i_j))
#     return bundle_star

In [5]:
max_iters = 100
tol = 1e-12


constraints_list = []

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        u_i = model.addVars(I, lb= 0 ,ub= GRB.INFINITY, name="utilities")
        p_j = model.addVars(J, lb= 0 ,ub= GRB.INFINITY, name="prices")

        model.setObjective( u_i.sum() + p_j.sum(), GRB.MINIMIZE)
        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}")
            ### Pricing problem
          
            B_star_i, _ = maximize_supermodular(np.arange(I), lambda_star, 300, 1,
                                                - np.ones(I)[:,None] * theta_solution[None, I:]) 
            reduced_cost_i = []
            for i in range(I):
                  reduced_cost_i.append(φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1,-1))[0] @ lambda_star 
                                - theta_solution[i] - theta_solution[I + np.where(B_star_i[i])[0] ].sum() )
            reduced_cost_i = np.array(reduced_cost_i)

            # stop if certificate holds
            print(f"reduced cost: {np.max(reduced_cost_i)}")
            if np.max(reduced_cost_i) <= tol:
                primal_solution = np.array(model.x)
                dual_solution = np.array(model.pi)
                print("DONE!")
                break
            
            ### Master problem

            model.addConstrs((u_i[i] + gp.quicksum(p_j[j] for j in np.where(B_star_i[i])[0])>= 
                            φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1,-1))[0] @ lambda_star 
                            for i in range(I)), name="constraint_batch")
            
            constraints_list.append(B_star_i)

            # Optimize the model
            u_i.start = reduced_cost_i + theta_solution[:I]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0
reduced cost: 372.4209340368052
##############
ITER: 1
reduced cost: 233.84727223929275
##############
ITER: 2


KeyboardInterrupt: 

In [6]:
theta_solution[:I]

array([ 0.        ,  0.        ,  0.        ,  9.83181225,  0.        ,
        0.        ,  4.30370916,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.96030218,
        8.40897528,  5.49174899,  0.        ,  0.        , 12.33122507,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [7]:
binding_constraints = np.array(constraints_list).reshape(-1,J)[dual_solution >0]
index = np.kron(np.ones(iter, dtype= int),np.arange(I))
matching = np.zeros((I ,J), dtype=bool)
matching[index[dual_solution > 0]]= binding_constraints

In [8]:
matching.sum(0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])